In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [2]:
model_id = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
label = pipeline('sentiment-analysis', model = model_id)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }


In [4]:
#web scrape session set up
url = 'https://financialpost.com/news/economy/canada-inflation-rate-higher-than-expected'
response = requests.get(url, headers=HEADERS)
page_content = response.text
doc = BeautifulSoup(page_content, 'html.parser')

In [5]:
#scrape trending news titles
header = []
news_elements = doc.find_all('h3', {"class" : 'article-card__headline m-0 article-card__headline--clamp-3'})
news_header = [] 
for i in range (len(news_elements)):
    news_header.append(news_elements[i].text)
news_header_df = pd.DataFrame({'News_Header': news_header})


In [6]:
#sentiment analysis on the headers
classification = []
for i in range (len(news_header_df.News_Header)):
    classification.append(label(news_header_df.News_Header[i]))

labels = []
for i in range (len(classification)):
    labels.append(classification[i][0]['label'])

In [7]:
news_header_df['sentiment_analysis'] = labels

In [8]:
news_header_df

,News_Header,sentiment_analysis
0,Canadian finds lithium eureka moment in a bag...,neutral
1,Posthaste: Why Bank of Canada hike is not a d...,neutral
2,Matthew Lau: Polls suggest Trudeau Liberals a...,negative
3,Ex-Bank of Canada governor Poloz sees cracks ...,negative
4,Need a charge? Canada’s $5-billion battery pl...,neutral


In [ ]:
vf-content-text vf-comment__content-editor

In [51]:
#scrape comments for news
comments = []
#comments_elements = doc.find_all('div', {"class" : 'vf-content-text vf-comment__content-editor'})
#comments_elements = doc.find_all('div', {"class" : 'all-comments'})
comments_elements = doc.find_all('div',{"class":'article-content__widget-group article-comment'})

In [ ]:
comments_elements

In [23]:
comments = []
comments_elements = doc.find_all('div', {"class" : 'vf-content-focus-container__contents'})
comments_elements
, {"class" : 'vf3-conversations-list--comments vf3-conversations-list vf3-conversations-list--no-separator'}
vf3-conversations-list--comments--list

[]

In [ ]:
https://stackoverflow.com/questions/65423418/web-scraping-div-class-not-found
https://stackoverflow.com/questions/69034436/web-scraping-using-beautifulsoup-selenium-unable-to-pull-div-class-using-find-or
https://www.reddit.com/r/learnpython/comments/lte011/webscraping_ul_and_li_tags_with_no_class/?rdt=57370&onetap_auto=true
